In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold 
from sklearn.preprocessing import LabelEncoder

In [2]:
train = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-2/train.csv')
test = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-2/test.csv')
subm = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-2/submission.csv')

In [3]:
le = LabelEncoder()
def FuncLabelEncoder(df):
    for c in df.columns:
        if df.dtypes[c] == object:
            le.fit(df[c].astype(str))
            df[c] = le.transform(df[c].astype(str))
    return df

In [4]:
train.drop(columns=['Id'], axis=1, inplace=True)

train['Province_State'] = train['Province_State'].fillna(-1)
test['Province_State'] = test['Province_State'].fillna(-1)

train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(train['Date'])

train['Date'] = train['Date'].astype('int64')
test['Date'] = test['Date'].astype('int64')

In [5]:
train = FuncLabelEncoder(train)
test = FuncLabelEncoder(test)

In [6]:
features = ['Province_State', 'Country_Region', 'Date']
target = 'ConfirmedCases'
target2 = 'Fatalities'

Random Forest

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,cross_val_predict

In [8]:
X1 = train[features]
y = train[target]

In [9]:
def rmsle(estimator, X, y):
    p = estimator.predict(X)
    return np.sqrt(mean_squared_error(np.log1p(y), np.log1p(p)))

In [11]:
rf_classif = RandomForestClassifier(n_estimators=120, random_state=1)
kf = KFold(n_splits=5, shuffle=True, random_state=1)

error = cross_val_score(rf_classif, X1, y, cv=kf,scoring=rmsle).mean()
print('RMSLE:', error)

RMSLE: 1.6516805332718332


In [10]:
rf_classif = RandomForestClassifier(n_estimators=110, random_state=0)
kf = KFold(n_splits=5, shuffle=True, random_state=1)

error = cross_val_score(rf_classif, X1, y, cv=kf,scoring=rmsle).mean()
print('RMSLE:', error)

RMSLE: 1.6472863717878283


In [ ]:
rf_classif.fit(train[features], train[target])

In [ ]:
predictions = rf_classif.predict(test[features])

In [ ]:
rf_classif.fit(train[features], train[target2])

In [ ]:
predictions1 = rf_classif.predict(test[features])

In [ ]:
submission = pd.DataFrame({'ForecastId':test['ForecastId'],'ConfirmedCases':predictions, 'Fatalities':predictions1})
submission.to_csv('submission.csv', index=False)

SVM

In [12]:
from sklearn import svm

In [14]:
clf_svm = svm.SVC()
error = cross_val_score(clf_svm, X1, y, cv=kf,scoring=rmsle).mean()
print('RMSLE:', error)

RMSLE: 2.9320156851344885


Gradient Descent

In [17]:
from sklearn.linear_model import SGDClassifier

clf_gd = SGDClassifier()
error = cross_val_score(clf_gd, X1, y, cv=kf,scoring=rmsle).mean()
print('RMSLE:', error)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


RMSLE: 4.397860523730896


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


Decision Tree

In [18]:
from sklearn import tree

clf_dec_tree = tree.DecisionTreeClassifier()

error = cross_val_score(clf_dec_tree, X1, y, cv=kf,scoring=rmsle).mean()
print('RMSLE:', error)

RMSLE: 0.8994099708195791


In [19]:
clf_dec_tree.fit(train[features], train[target])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
predictions = clf_dec_tree.predict(test[features])

In [ ]:
clf_dec_tree.fit(train[features], train[target2])

In [ ]:
predictions1 = clf_dec_tree.predict(test[features])

In [ ]:
submission = pd.DataFrame({'ForecastId':test['ForecastId'],'ConfirmedCases':predictions, 'Fatalities':predictions1})
submission.to_csv('submission.csv', index=False)